In [1]:
import numpy as np
import cv2 as cv
import os
import YOLO as yolo_class
import Unet as unet_class
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
yolo = yolo_class.YOLO(classesFile="coco.names",modelWeights="yolov3.weights",modelConfiguration="yolov3.cfg")
unet = unet_class.Unet('third_training.h5')
outputFile = "test.avi";
videoFile = 'Syrischer Student in KIT.mp4';
cap = cv.VideoCapture(videoFile)
print(cap.isOpened())
Outputfile = videoFile[:-4]
vid_writer = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc('M','J','P','G'), 30, (int(cap.get(3)), int(cap.get(4))))         
# vid_writer = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc(*'XVID'), 30, (640, 480))    

while cv.waitKey(1) < 0:
    
    # get frame from the video
    hasFrame, frame = cap.read()
    
    # Stop the program if reached end of video
    if not hasFrame:
        print("Done processing !!!")
        print("Output file is stored as ", outputFile)
        cv.waitKey(3000)
        # Release device
        cap.release()
        break
    frame_test = frame.copy()

    unet.getMask(frame_test)
    
    blob = cv.dnn.blobFromImage(frame, 1/255, (yolo.inpWidth, yolo.inpHeight), [0,0,0], 1, crop=False)
    yolo.net.setInput(blob)
    outs = yolo.net.forward(yolo.getOutputsNames())
    yolo.postprocess(frame, outs)
    
    cv.imshow('yolo', frame) 
    frame = cv.resize(frame,(640, 480)) 
    
    frame = unet.drawContours(frame)
    frame = cv.resize(frame, (int(cap.get(3)), int(cap.get(4))))
    cv.imshow('semantic', frame)
    frame_write = frame.copy()

    vid_writer.write(frame_write.astype(np.uint8))

True
1/1 [==============================] - 0s 269ms/step


In [29]:
cv.destroyAllWindows()